In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Aug 22 00:48:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!pip -V

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [4]:
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 1.3 MB 47.6 MB/s 
     |████████████████████████████████| 101 kB 43.3 MB/s 
     |████████████████████████████████| 6.6 MB 30.1 MB/s 


In [5]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 49.5 MB/s 
     |████████████████████████████████| 115 kB 43.7 MB/s 
     |████████████████████████████████| 127 kB 66.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [6]:
import os

import logging
from types import SimpleNamespace
from pathlib import Path
from datetime import datetime
import math

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
from transformers import TrainingArguments, Trainer
from tqdm import tqdm
from scipy.special import softmax
from IPython.core.display import display, HTML

from transformers import DataCollatorWithPadding
from datasets import Dataset, load_metric


# From this Gist: https://gist.github.com/ihoromi4/b681a9088f348942b01711f251e5f964
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
import torch
from torch import nn
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers.models.deberta_v2.modeling_deberta_v2 import ContextPooler
from transformers.models.deberta_v2.modeling_deberta_v2 import StableDropout
from transformers.modeling_outputs import TokenClassifierOutput

In [8]:
device = 'cpu'
if torch.cuda.is_available:
    device = 'cuda'
print(device)

cuda


In [9]:
labels = ['Adequate', 'Effective', 'Ineffective']

In [10]:
base_path = 'drive/MyDrive/Comps/FeedbackPrize'

In [11]:
data_2021_path = 'drive/MyDrive/Comps/FeedbackPrize/feedback-prize-2021'
data_2022_path = 'drive/MyDrive/Comps/FeedbackPrize/feedback-prize-effectiveness'

train_df = pd.read_csv(os.path.join(data_2022_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_2022_path, 'test.csv'))

## Using 2021 data predicting topics and join pred topics to 2022 data set

In [12]:
topic_pred_df = pd.read_csv('drive/MyDrive/Comps/FeedbackPrize/topics/topic_model_feedback.csv')
topic_pred_df = topic_pred_df.drop(columns={'prob'})
topic_pred_df = topic_pred_df.rename(columns={'id': 'essay_id'})

topic_meta_df = pd.read_csv('drive/MyDrive/Comps/FeedbackPrize/topics/topic_model_metadata.csv')
topic_meta_df = topic_meta_df.rename(columns={'Topic': 'topic', 'Name': 'topic_name'}).drop(columns=['Count'])
topic_meta_df.topic_name = topic_meta_df.topic_name.apply(lambda n: ' '.join(n.split('_')[1:]))

topic_pred_df = topic_pred_df.merge(topic_meta_df, on='topic', how='left')
topic_pred_df['essay_id'] = topic_pred_df.essay_id.apply(lambda x: x[6:])

train_df = train_df.merge(topic_pred_df, on='essay_id', how='left')

## Set Config

In [13]:
config = SimpleNamespace()

config.seed = 420
config.model_name = 'microsoft/deberta-v3-base'
config.output_path = Path('drive/MyDrive/Comps/FeedbackPrize/output')
config.input_path = Path('drive/MyDrive/Comps/FeedbackPrize/feedback-prize-effectiveness')

config.n_folds = 4
config.lr = 1e-5
config.weight_decay = 0.01
config.epochs = 4
config.batch_size = 16
config.gradient_accumulation_steps = 1
config.warm_up_ratio = 0.1
config.max_len = 384
config.hidden_dropout_prob = 0.1
config.label_smoothing_factor = 0.
config.eval_per_epoch = 2

logging.disable(logging.WARNING)

seed_everything(config.seed)

## Add 2021 data

In [15]:
train_2021_preds_df = pd.read_csv(os.path.join(config.output_path, 'train_2021_preds.csv'))
train_2021_preds_df = train_2021_preds_df[train_2021_preds_df.in_2022 == False]

In [16]:
num_essays = 10
train_2021_preds_df['label_prob'] = train_2021_preds_df[labels].max(axis=1)
confident_essays = train_2021_preds_df[['essay_id', 'label_prob']].groupby('essay_id').mean().sort_values('label_prob', ascending=False)[:num_essays]
essay_ids = set(confident_essays.index)
train_2021_filt_df = train_2021_preds_df[train_2021_preds_df.essay_id.isin(essay_ids)].reset_index(drop=True)
train_2021_filt_df.shape

(10, 13)

In [17]:
#train_2021_filt_df

## Using CV

In [18]:
cv = StratifiedKFold(n_splits=config.n_folds, shuffle=True, random_state=config.seed)
train_df['fold'] = -1
for fold_num, (train_idxs, test_idxs) in enumerate(cv.split(train_df.index, train_df.discourse_effectiveness, train_df.essay_id)):
    train_df.loc[test_idxs, ['fold']] = fold_num

In [19]:
train_df.to_csv(config.output_path / 'train_folds.csv', index=False)

## Tokenizer

In [20]:
config.model_name

'microsoft/deberta-v3-base'

In [21]:
tokenizer = AutoTokenizer.from_pretrained(config.model_name, use_fast=True)
tokenizer.model_max_length = config.max_len

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [22]:
tokenizer

PreTrainedTokenizerFast(name_or_path='microsoft/deberta-v3-base', vocab_size=128000, model_max_len=384, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [23]:
def get_essay(essay_fns):
  essay_cache = {}

  output = []
  for essay_fn in essay_fns:
      if essay_fn not in essay_cache:
          essay_txt = open(essay_fn).read().strip().lower()
          essay_cache[essay_fn] = essay_txt
      output.append(essay_cache[essay_fn])

  return output

def tokenizer_func(x):
  return tokenizer(x["inputs"], get_essay(x['essay_fn']), truncation=True, max_length=config.max_len)

def add_inputs(df, basepath):
  df['essay_fn'] = basepath + '/' + df.essay_id + '.txt'
  df['inputs'] = df.discourse_type.str.lower() + ' ' + tokenizer.sep_token + ' ' + df.topic_name + ' ' + tokenizer.sep_token + ' ' + df.discourse_text.str.lower()
  return df

In [24]:
train_df = add_inputs(train_df, str(config.input_path / 'train'))
train_2021_filt_df = add_inputs(train_2021_filt_df, 'drive/MyDrive/Comps/FeedbackPrize/feedback-prize-2021/train')

## Model

In [25]:
def get_dropouts(num, start_prob, increment):
    return [StableDropout(start_prob + (increment * i)) for i in range(num)]  

class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, backbone):
        super(CustomModel, self).__init__()
        
        self.model = backbone
        self.config = self.model.config
        self.num_labels = self.config.num_labels

        # self.pooler = ContextPooler(self.config)
        self.pooler = MeanPooling()
        
        self.classifier = nn.Linear(self.config.hidden_size, self.num_labels)
    
        self.dropouts = get_dropouts(num=5, start_prob=config.hidden_dropout_prob - 0.02, increment=0.01)
    
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None
    ):
        outputs = self.model.deberta(
            input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        encoder_layer = outputs[0]
        pooled_output = self.pooler(encoder_layer, attention_mask)
                      
        # Multi-sample dropout.
        num_dps = float(len(self.dropouts))
        for ii, drop in enumerate(self.dropouts):
            if ii == 0:
                logits = (self.classifier(drop(pooled_output)) / num_dps)
            else:
                logits += (self.classifier(drop(pooled_output)) / num_dps)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            logits = logits.view(-1, self.num_labels)
            loss = loss_fn(logits, labels.view(-1))

        output = (logits,) + outputs[1:]

        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)

In [26]:
def get_backbone_config():
    model_config = AutoConfig.from_pretrained(config.model_name, num_labels=3)
    model_config.hidden_dropout_prob = config.hidden_dropout_prob
    return model_config

In [27]:
def get_model():
    model_config = get_backbone_config()

    model = AutoModelForSequenceClassification.from_pretrained(
        config.model_name,
        config=model_config,
    )
    # print(model)
    return CustomModel(model)

In [28]:
backbone_config = get_backbone_config()
backbone_config.save_pretrained(os.path.join(base_path, 'backbone_config'))

In [29]:
model = get_model()

## Training

In [30]:
# add metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [31]:
# train_df = train_df.sample(n=150)
# config.epochs = 1

In [32]:
def do_fold(fold_num):
    train_data  = train_df.query(f'fold != {fold_num}').reset_index(drop=True)

    val_data  = train_df.query(f'fold == {fold_num}').reset_index(drop=True)
    
    # Add 2021 to train data.
    train_data = pd.concat([train_data, train_2021_filt_df[['inputs', 'essay_fn', 'discourse_effectiveness']]]).sample(frac=1., random_state=config.seed).reset_index(drop=True)
    print(f'Train data size: {train_data.shape}')

    train_dataset = Dataset.from_pandas(train_data[['inputs', 'essay_fn', 'discourse_effectiveness']]).rename_column('discourse_effectiveness', 'label').class_encode_column("label")
    val_dataset = Dataset.from_pandas(val_data[['inputs', 'essay_fn', 'discourse_effectiveness']]).rename_column('discourse_effectiveness', 'label').class_encode_column("label")

    train_tok_dataset = train_dataset.map(tokenizer_func, batched=True, remove_columns=('inputs', 'essay_fn'))
    val_tok_dataset = val_dataset.map(tokenizer_func, batched=True, remove_columns=('inputs', 'essay_fn'))

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest')

    num_steps = len(train_data) / config.batch_size / config.gradient_accumulation_steps
    eval_steps = num_steps // config.eval_per_epoch
    print(f'Num steps: {num_steps}, eval steps: {eval_steps}')

    args = TrainingArguments(
        output_dir=config.output_path,
        learning_rate=config.lr,
        warmup_ratio=config.warm_up_ratio,
        lr_scheduler_type='cosine',
        fp16=True,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=config.batch_size * 2,
        num_train_epochs=config.epochs,
        weight_decay=config.weight_decay,
        # report_to="wandb",

        evaluation_strategy='steps',
        eval_steps=eval_steps, 
        save_strategy='steps',
        save_steps=eval_steps,
        
        load_best_model_at_end=True,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        label_smoothing_factor=config.label_smoothing_factor,
        save_total_limit=3  # Prevents running out of disk space.
    )

    model = get_model()

    trainer = Trainer(
        model,
        args,
        train_dataset=train_tok_dataset,
        eval_dataset=val_tok_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()
    
    trainer.save_model(config.output_path / f'fold_{fold_num}')
    
    outputs = trainer.predict(val_tok_dataset)

    val_data[labels] = softmax(outputs.predictions, axis=1)
    
    !rm -rf {config.output_path / 'checkpoint'}*
    
    return val_data

In [33]:
val_preds_df = pd.DataFrame()

val_data = do_fold(0)

val_preds_df = pd.concat([val_preds_df, val_data])

Train data size: (27583, 10)


Casting to class labels:   0%|          | 0/28 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/10 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

Num steps: 1723.9375, eval steps: 861.0


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss,Validation Loss,Accuracy
861,0.848700,0.755529,0.675479
1722,0.717800,0.682631,0.703764
2583,0.646400,0.637980,0.724217
3444,0.626400,0.598909,0.742493
4305,0.533400,0.700011,0.710400
5166,0.526600,0.594407,0.744778
6027,0.467100,0.595416,0.753481
6888,0.460700,0.596894,0.754678


In [34]:
for fold in range(1, config.n_folds):
    val_data = do_fold(fold)
    val_preds_df = pd.concat([val_preds_df, val_data])

Train data size: (27584, 10)


Casting to class labels:   0%|          | 0/28 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/10 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

Num steps: 1724.0, eval steps: 862.0


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss,Validation Loss,Accuracy
862,0.850300,0.761561,0.666630
1724,0.721700,0.770637,0.651507
2586,0.643600,0.634967,0.723751
3448,0.626600,0.623792,0.725166
4310,0.539700,0.633583,0.729627
5172,0.540400,0.620025,0.736481
6034,0.479200,0.623948,0.733326
6896,0.459400,0.627190,0.734632


In [35]:
val_preds_df.drop(columns=['inputs']).to_csv(config.output_path / 'val_preds.csv', index=False)
val_preds_df = pd.read_csv(config.output_path / 'val_preds.csv')